In [ ]:
import subprocess
import re

def run_bombardier(url, duration, connections):
    command = f'bombardier -d {duration} -c {connections} {url}'
    output = subprocess.check_output(command, shell=True, text=True)
    return output

def identify_anomalies(output):
    # Parse the output to identify anomalies
    results = re.findall(r'\b\d+\b', output)
    results = list(map(int, results))
    avg_throughput = sum(results) / len(results)
    max_throughput = max(results)
    min_throughput = min(results)
    anomalies = [result for result in results if result < min_throughput * 0.8 or result > max_throughput * 1.2]
    return avg_throughput, min_throughput, max_throughput, anomalies

if __name__ == "__main__":
    url = "http://90.243.14.148/api/v1/users"  # Replace with your API URL
    duration = '10s'  # Duration of the test in seconds
    connections = 60000  # Number of connections
    output = run_bombardier(url, duration, connections)
    avg_throughput, min_throughput, max_throughput, anomalies = identify_anomalies(output)
    print(f"Average Throughput: {avg_throughput}")
    print(f"Min Throughput: {min_throughput}")
    print(f"Max Throughput: {max_throughput}")
    if anomalies:
        print("Anomalies detected:")
        for anomaly in anomalies:
            print(anomaly)
    else:
        print("No anomalies detected.")

In [4]:
import subprocess
import time

base_concurrency = 100
base_requests = 100
concurrency_limit = 110


def run_script(command):
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    return result.stdout

def parse_output(output):
    lines = output.split('\n')
    print(lines)
    return lines


for i in range(100000):
    # command = f"bombardier -c 100 -n 100 http://192.168.1.35:9999/api/v1/users"
    command = f"nload"
    test_output = run_script(command)
    parse_output(test_output)
    time.sleep(5)
    print(f"Test {i} completed")

KeyboardInterrupt: 

In [ ]:
command = f'bombardier -d 10s -c 100000 http://192.168.1.35:9999/api/v1/users'
output = subprocess.check_output(command, shell=True, text=True)
print(output)

In [ ]:
import psutil
import time
import os
import pandas as pd

UPDATE_DELAY = 1 # in seconds

def get_size(bytes):
    """
    Returns size of bytes in a nice format
    """
    for unit in ['', 'K', 'M', 'G', 'T', 'P']:
        if bytes < 1024:
            return f"{bytes:.2f}{unit}B"
        bytes /= 1024

# get the network I/O stats from psutil on each network interface
# by setting `pernic` to `True`
io = psutil.net_io_counters(pernic=True)

while True:
    # sleep for `UPDATE_DELAY` seconds
    time.sleep(UPDATE_DELAY)
    # get the network I/O stats again per interface 
    io_2 = psutil.net_io_counters(pernic=True)
    # initialize the data to gather (a list of dicts)
    data = []
    for iface, iface_io in io.items():
        # new - old stats gets us the speed
        upload_speed, download_speed = io_2[iface].bytes_sent - iface_io.bytes_sent, io_2[iface].bytes_recv - iface_io.bytes_recv
        data.append({
            "iface": iface, "Download": get_size(io_2[iface].bytes_recv),
            "Upload": get_size(io_2[iface].bytes_sent),
            "Upload Speed": f"{get_size(upload_speed / UPDATE_DELAY)}/s",
            "Download Speed": f"{get_size(download_speed / UPDATE_DELAY)}/s",
        })
    # update the I/O stats for the next iteration
    io = io_2
    # construct a Pandas DataFrame to print stats in a cool tabular style
    df = pd.DataFrame(data)
    # sort values per column, feel free to change the column
    df.sort_values("Download", inplace=True, ascending=False)
    # clear the screen based on your OS
    os.system("cls") if "nt" in os.name else os.system("clear")
    # print the stats
    print(df.to_string())